In [ ]:
%use kotlin-dl
%use krangl

In [ ]:
import org.jetbrains.kotlinx.dl.api.core.history.EpochTrainingEvent
import org.jetbrains.kotlinx.dl.api.core.history.TrainingHistory

In [ ]:
fun normalize(df: DataFrame): DataFrame {
    val normalizedCols = mutableListOf<DataCol>()
    for (col in df.cols) {
        val min = col.min()!!
        val max = col.max()!!
        val normalized = (col - min) / (max - min)
        normalizedCols.add(normalized)
    }
    val normalizedDF = dataFrameOf(*normalizedCols.toTypedArray())
    normalizedDF.setNames(*df.names.toTypedArray())
    return normalizedDF
}

In [ ]:
fun getXy(
    df: DataFrame, 
    label: String = "quality"
): Pair<Array<FloatArray>, FloatArray> {
    val features = df.remove(label)
    val nFeatures = features.ncol
    val normalizedFeatures = normalize(features)
    val columnsArray = normalizedFeatures.toFloatMatrix()
    
    val X = Array(features.nrow) { FloatArray(nFeatures) }

    for (col in 0 until nFeatures) {
        for (row in 0 until df.nrow) {
            X[row][col] = columnsArray[col][row]
        }
    }
    
    val labels = df.get(label).toDoubles().filterNotNull().map { it.toFloat() }
    
    val y = labels.toFloatArray()
    return Pair(X, y)
}

In [ ]:
val dfTrain = DataFrame.readCSV("data/winequality-white-train.csv")
val dfValid = DataFrame.readCSV("data/winequality-white-val.csv")

In [ ]:
val (X, y) = getXy(dfTrain)
val (XValid, yValid) = getXy(dfValid)

In [ ]:
X[0].forEach { print("$it ") }
print(y[0])

In [ ]:
val trainDataset = OnHeapDataset.create(X, y)
val validDataset = OnHeapDataset.create(XValid, yValid)

In [ ]:
class PrintingCallback : Callback() {
    override fun onEpochEnd(epoch: Int, event: EpochTrainingEvent, logs: TrainingHistory) {
        println("Epoch: $epoch - loss: ${event.lossValue} - val loss: ${event.valLossValue}")
    }
}

In [ ]:
val model = Sequential.of(
    Input(11),
    Dense(8),
    Dense(8),
    Dense(8),
    Dense(8),
    Dense(8),
    Dense(8),
    Dense(8),
    Dense(1, activation = Activations.Linear)
)


In [ ]:
model.compile(
    optimizer = Adam(0.001f),
    loss = Losses.MAE,
    metric = Metrics.MAE,
    callback = PrintingCallback(),
)

In [ ]:
model.summary().print()

In [ ]:
model.fit(
    trainingDataset = trainDataset,
    validationDataset = validDataset,
    epochs = 250,
    trainBatchSize = 32,
    validationBatchSize = 1024
)    

In [ ]:
val result = model.evaluate(validDataset)
println(result.lossValue)